In [1]:
import gym
import random
import numpy as np
# import tensorflow as tf
#import time
from gym.envs.registration import register
from IPython.display import clear_output
from collections import deque

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() # use Tensorflow v1

2022-07-11 19:31:02.859634: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-11 19:31:02.862753: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/fernandagitai/Documents/ufal/ML/ML-Reinforcement/venv/lib/python3.8/site-packages/cv2/../../lib64:
2022-07-11 19:31:02.862763: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/fernandagitai/Documents/ufal/ML/ML-Reinforcement/venv/lib/python3.8/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the 

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
tf.compat.v1.disable_eager_execution() 

In [4]:
class Agent():
  def __init__(self, env):

    self.is_discrete = type(env.action_space) == gym.spaces.discrete.Discrete

    if self.is_discrete:
      self.action_size = env.action_space.n

    else:
      self.action_low = env.action_space.low
      self.action_high = env.action_space.high
      self.action_shape = env.action_space.shape
  
  def get_action(self, state):

    if self.is_discrete:
      #Acao randomica
      action = random.choice(range(self.action_size))
    else:
      action = np.random.uniform(self.action_low, self.action_high, self.action_shape)

    return action

In [5]:
class QNAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.001):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("State size:", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()

        self.sess = tf.compat.v1.Session()
        self.sess.run(tf.compat.v1.global_variables_initializer())

        self.replay_buffer = deque(maxlen=10000)
        
    def build_model(self):
        tf.compat.v1.reset_default_graph()
        self.state_in = tf.compat.v1.placeholder(tf.int32, [None])
        self.action_in = tf.compat.v1.placeholder(tf.int32, [None])
        self.target_in = tf.compat.v1.placeholder(tf.float32, [None])

        self.state = tf.one_hot(self.state_in, depth = self.state_size)
        self.action = tf.one_hot(self.action_in, depth = self.action_size)

        self.q_state = tf.compat.v1.layers.dense(self.state, units = self.action_size, name = "q_table")
        self.q_action = tf.reduce_sum(tf.multiply(self.q_state, self.action), axis = 1)

        self.loss = tf.reduce_sum(tf.square(self.target_in - self.q_action))
        self.optimizer = tf.compat.v1.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        
    def get_action(self, state):
        # q_state = self.q_table[state]
        q_state = self.sess.run(self.q_state, feed_dict={self.state_in: [state]})
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience, batch_size=50):
        self.replay_buffer.append(experience)
        samples = random.choices(self.replay_buffer, k=batch_size)
        state, action, next_state, reward, done = (list(col) for col in zip(experience, *samples))
        
        q_next = self.sess.run(self.q_state, feed_dict={self.state_in: next_state})
        q_next[done] = np.zeros([self.action_size])
        q_target = reward + self.discount_rate * np.max(q_next, axis =1)

        feed = {self.state_in: state, self.action_in: action, self.target_in: q_target}
        self.sess.run(self.optimizer, feed_dict=feed)
        
        if experience[4]:
            self.eps = self.eps * 0.99

    def __del__(self):
        self.sess.close()

In [13]:
env = gym.make('FrozenLake-v1', is_slippery=True, map_name='4x4')
# env = gym.make('FrozenLake-v1', is_slippery=True, map_name='8x8')
agent = QNAgent(env)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

State size: 16
Observation space: Discrete(16)
Action space: Discrete(4)


/tmp/ipykernel_121276/2122706099.py:27: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  self.q_state = tf.compat.v1.layers.dense(self.state, units = self.action_size, name = "q_table")


In [15]:
total_reward = 0
for ep in range(300):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state,action,next_state,reward,done))
        state = next_state
        total_reward += reward
        print("s:", state, "a:", action)
        print("Episode: {}, Total reward: {}, eps: {}".format(ep,total_reward,agent.eps))
        env.render()

        with tf.compat.v1.variable_scope("q_table", reuse = True):
            weights = agent.sess.run(tf.compat.v1.get_variable("kernel"))
            print(weights)
            # scope.reuse_variables()
        #print(agent.q_table)
        clear_output(wait=True)
env.close()

s: 7 a: 2
Episode: 99, Total reward: 12.0, eps: 0.13397967485796175
[[-0.02066821 -0.07625312 -0.09003542 -0.11305766]
 [-0.11159261 -0.17920475 -0.18715    -0.11419608]
 [-0.0239376  -0.07559817 -0.09798582 -0.11070468]
 [-0.12021001 -0.1841866  -0.2421692  -0.11428628]
 [-0.00133055 -0.1674994  -0.125471   -0.16606429]
 [ 0.01337498 -0.01913208  0.2875983   0.06674117]
 [-0.13078278 -0.23089011 -0.07221932 -0.21596447]
 [ 0.2139219  -0.40660426 -0.30039442 -0.17153847]
 [-0.03353601 -0.08512703 -0.088287   -0.0344838 ]
 [ 0.04966659  0.11832376 -0.04730106 -0.05061626]
 [ 0.21794666  0.14041428 -0.05625635 -0.15575044]
 [-0.1300005  -0.30310747 -0.11734465  0.4093197 ]
 [-0.10793218  0.3925612  -0.33868754 -0.04820457]
 [ 0.10419398 -0.02120613  0.21731031 -0.09222919]
 [ 0.31674993  0.42697054  0.25841743 -0.00764201]
 [-0.20710576 -0.29734662  0.22385395  0.47462702]]
